# Install and Import libraries

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 101.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.7 MB/s eta 0:00:00


In [ ]:
!pip install datasets -q
!pip install tokenizers -q
!pip install transformers -q
!pip install seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
''' import libraries '''
import pandas as pd

from transformers import AutoTokenizer, AutoModelForTokenClassification #Del
from transformers import pipeline #Del

# import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification #Del BertConfig

import re

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


# Data exploration
1. Sentence construction around each entity, providing contextual information
2. Various variations associated with each entity
3. Frequency of each entity

In [ ]:
df = pd.read_csv('PII Train Data.csv')
df

FileNotFoundError: ignored

In [ ]:
sentences = df['Text'].tolist()
sentences[0]

NameError: ignored

In [ ]:
unique_labels = df['Labels'].unique().tolist()
for label in unique_labels:
  selected_list = df[df['Labels']==label]['PII'].tolist()
  print (f'{label} : {selected_list}')

NameError: ignored

# Transform dataset
Prepare the dataset by transforming it into the required format for tokenization

In [ ]:
df = df.loc[df['Labels'] != 'None'] # Next iter-consider these labels as well

In [ ]:
df.columns

Index(['Text', 'Labels', 'PII'], dtype='object')

In [ ]:
# transform dataset
df['tokens'] = df['Text'].apply(lambda x: x.split())
df['tags'] = df.apply(lambda row: ','.join([row['Labels'] if word in row['PII'] else 'O' for word in row['Text'].split()]), axis=1)
df

<ipython-input-14-4ce7f55cd52e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['Text'].apply(lambda x: x.split())
<ipython-input-14-4ce7f55cd52e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df.apply(lambda row: ','.join([row['Labels'] if word in row['PII'] else 'O' for word in row['Text'].split()]), axis=1)


,Text,Labels,PII,tokens,tags
0,That goal above agent young well none. 00779 N...,Address,00779 Nicole Mountain Apt. 347 North Veronicab...,"[That, goal, above, agent, young, well, none.,...","O,O,O,O,O,O,O,Address,Address,Address,Address,..."
1,Usually itself strong onto way own she. Tough ...,Address,Suite 786,"[Usually, itself, strong, onto, way, own, she....","O,O,O,O,O,O,O,O,O,O,O,O,O,O,Address,Address,O"
2,Either author agency boy. Born it own past 007...,Address,00779 Nicole Mountain Apt. 347 North Veronicab...,"[Either, author, agency, boy., Born, it, own, ...","O,O,O,O,O,O,O,O,Address,Address,Address,Addres..."
3,Remain guess want home fire center can opportu...,Address,Suite 786,"[Remain, guess, want, home, fire, center, can,...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,Address,Address,O,..."
4,Teacher green level sport officer area. Struct...,Address,5779 Patterson Shore Suite 089,"[Teacher, green, level, sport, officer, area.,...","O,O,O,O,O,O,O,Address,Address,Address,Address,..."
...,...,...,...,...,...
65,Occur across mission ever interview land other...,SSN,493-60-3658,"[Occur, across, mission, ever, interview, land...","O,O,O,O,O,O,O,O,O,O,O,SSN,O,O,O,O,O"
66,Cut fall history 397-25-6084 end. Dream car na...,SSN,397-25-6084,"[Cut, fall, history, 397-25-6084, end., Dream,...","O,O,O,SSN,O,O,O,O,O,O,O,O"
67,Low you form after put gas. Soon maybe like. O...,SSN,494-82-9717,"[Low, you, form, after, put, gas., Soon, maybe...","O,O,O,O,O,O,O,O,O,O,O,O,SSN,O,O,O,O,O,O,O,O"
68,Hour blue city watch become parent travel. 368...,SSN,368 63 1719,"[Hour, blue, city, watch, become, parent, trav...","O,O,O,O,O,O,O,SSN,SSN,SSN,O,O,O,O,O,O,O,O"


# NER : Name Entity
/added/<br>
Additional data for 'Name' entity based on initial iteration predictions of the model

In [ ]:
ner_df = pd.read_csv('ner.csv')
ner_df.head()

,text,labels
0,Thousands of demonstrators have marched throug...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Iranian officials say they expect to get acces...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...
2,Helicopter gunships Saturday pounded militant ...,O O B-tim O O O O O B-geo O O O O O B-org O O ...
3,They left after a tense hour-long standoff wit...,O O O O O O O O O O O
4,U.N. relief coordinator Jan Egeland said Sunda...,B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...


In [ ]:
# subset_ner_df = ner_df[ner_df['labels'].apply(lambda lst: any(search_string in lst for search_string in ['I-per']))]
subset_ner_df = ner_df[ner_df['labels'].str.contains('I-per')]
subset_ner_df

,text,labels
4,U.N. relief coordinator Jan Egeland said Sunda...,B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...
5,Mr. Egeland said the latest figures show 1.8 m...,B-per I-per O O O O O O O O O O O O O O O O O ...
10,"In Beirut , a string of officials voiced their...",O B-geo O O O O O O O O O O O O B-org I-org O ...
12,Lebanon has suffered a series of bombings sinc...,B-geo O O O O O O O O O O O B-tim O O O B-per ...
13,Syria is widely accused of involvement in his ...,B-geo O O O O O O O O O O B-tim O O O O O B-ge...
...,...,...
47935,The Associated Press quotes Afghan President H...,B-org I-org I-org O B-gpe B-per I-per I-per O ...
47947,Democrat Tom Daschle contradicts President Bus...,O B-per I-per O O B-per O O O B-org O O O O O ...
47949,In an opinion piece in the Washington Post Fri...,O O O O O O B-org I-org B-tim O B-per I-per O ...
47950,President Bush last week confirmed he secretly...,B-per I-per O O O O O O O B-org I-org I-org O ...


In [ ]:
# Function to modify the string
def modify_string(string):
    string_list = string.split()
    modified_list = ['Name' if item == 'I-per' else 'O' for item in string_list]
    modified_string = ','.join(modified_list)
    return modified_string

# Apply the function to the column
subset_ner_df['labels'] = subset_ner_df['labels'].apply(modify_string)

subset_ner_df

<ipython-input-17-fd319f164b00>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_ner_df['labels'] = subset_ner_df['labels'].apply(modify_string)


,text,labels
4,U.N. relief coordinator Jan Egeland said Sunda...,"O,O,O,O,Name,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O..."
5,Mr. Egeland said the latest figures show 1.8 m...,"O,Name,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O..."
10,"In Beirut , a string of officials voiced their...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
12,Lebanon has suffered a series of bombings sinc...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,Name,O,O..."
13,Syria is widely accused of involvement in his ...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,Name,O..."
...,...,...
47935,The Associated Press quotes Afghan President H...,"O,O,O,O,O,O,Name,Name,O,O,O,O,O,O,O,O,O,O"
47947,Democrat Tom Daschle contradicts President Bus...,"O,O,Name,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O..."
47949,In an opinion piece in the Washington Post Fri...,"O,O,O,O,O,O,O,O,O,O,O,Name,O,O,O,O,O,O,O,O,O,O..."
47950,President Bush last week confirmed he secretly...,"O,Name,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"


In [ ]:
subset_ner_df = subset_ner_df[0:100]

In [ ]:
subset_ner_df['Labels'] = 'Name'
subset_ner_df['PII'] = np.nan
subset_ner_df['tokens'] = np.nan

new_column_names = {'text': 'Text',
                    'labels': 'tags'}

# Rename the columns
subset_ner_df = subset_ner_df.rename(columns=new_column_names)
subset_ner_df

<ipython-input-19-50445737af4d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_ner_df['Labels'] = 'Name'
<ipython-input-19-50445737af4d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_ner_df['PII'] = np.nan
<ipython-input-19-50445737af4d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

,Text,tags,Labels,PII,tokens
4,U.N. relief coordinator Jan Egeland said Sunda...,"O,O,O,O,Name,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O...",Name,NaN,NaN
5,Mr. Egeland said the latest figures show 1.8 m...,"O,Name,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O...",Name,NaN,NaN
10,"In Beirut , a string of officials voiced their...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...",Name,NaN,NaN
12,Lebanon has suffered a series of bombings sinc...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,Name,O,O...",Name,NaN,NaN
13,Syria is widely accused of involvement in his ...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,Name,O...",Name,NaN,NaN
...,...,...,...,...,...
402,The head of Iran 's Atomic Energy Organization...,"O,O,O,O,O,O,O,O,O,O,Name,O,O,O,O,O,O,O,O,O,O,O",Name,NaN,NaN
403,But Iraq 's Prime Minister Ibrahim al-Jaafari ...,"O,O,O,O,O,O,Name,O,O,O,O,O,O,O,O,O",Name,NaN,NaN
408,VOA 's Carol Pearson reports .,"O,O,O,Name,O,O",Name,NaN,NaN
419,Thousands of motorcycle riders are in Washingt...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,Name,O,O,O,O,O,O,O...",Name,NaN,NaN


In [ ]:
df_appended = df.append(subset_ner_df)
df_appended.reset_index(drop=True,inplace=True)
df_appended

<ipython-input-20-df7639deb281>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_appended = df.append(subset_ner_df)


,Text,Labels,PII,tokens,tags
0,That goal above agent young well none. 00779 N...,Address,00779 Nicole Mountain Apt. 347 North Veronicab...,"[That, goal, above, agent, young, well, none.,...","O,O,O,O,O,O,O,Address,Address,Address,Address,..."
1,Usually itself strong onto way own she. Tough ...,Address,Suite 786,"[Usually, itself, strong, onto, way, own, she....","O,O,O,O,O,O,O,O,O,O,O,O,O,O,Address,Address,O"
2,Either author agency boy. Born it own past 007...,Address,00779 Nicole Mountain Apt. 347 North Veronicab...,"[Either, author, agency, boy., Born, it, own, ...","O,O,O,O,O,O,O,O,Address,Address,Address,Addres..."
3,Remain guess want home fire center can opportu...,Address,Suite 786,"[Remain, guess, want, home, fire, center, can,...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,Address,Address,O,..."
4,Teacher green level sport officer area. Struct...,Address,5779 Patterson Shore Suite 089,"[Teacher, green, level, sport, officer, area.,...","O,O,O,O,O,O,O,Address,Address,Address,Address,..."
...,...,...,...,...,...
165,The head of Iran 's Atomic Energy Organization...,Name,NaN,NaN,"O,O,O,O,O,O,O,O,O,O,Name,O,O,O,O,O,O,O,O,O,O,O"
166,But Iraq 's Prime Minister Ibrahim al-Jaafari ...,Name,NaN,NaN,"O,O,O,O,O,O,Name,O,O,O,O,O,O,O,O,O"
167,VOA 's Carol Pearson reports .,Name,NaN,NaN,"O,O,O,Name,O,O"
168,Thousands of motorcycle riders are in Washingt...,Name,NaN,NaN,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,Name,O,O,O,O,O,O,O..."


In [ ]:
df = df_appended

In [ ]:
label2id = {k: v for v, k in enumerate(df.Labels.unique().tolist() + ["O"])}
id2label = {v: k for v, k in enumerate(df.Labels.unique().tolist() + ["O"])}
label2id

{'Address': 0,
 'CreditCardNumber': 1,
 'Email': 2,
 'Name': 3,
 'Phone_number': 4,
 'Plates': 5,
 'SSN': 6,
 'O': 7}

# Set configurations

In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 10
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10 # to avoid gradient explosion : changing the derivatives of the loss function to have a given vector norm when the L2 vector norm (sum of the squared values) of the gradient vector exceeds a threshold value
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') #For example, the `bert-case-uncased` tokenizer used in the example above has a vocabulary size of only 30,522 tokens.

# Initialize tokenizer and Split data

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.Text[index]
        word_labels = self.data.tags[index]
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [ ]:
train_size = 0.8
train_dataset = df.sample(frac=train_size,random_state=200) # stratify
test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (170, 5)
TRAIN Dataset: (136, 5)
TEST Dataset: (34, 5)


# Create dataloaders

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
training_set[0]['ids']

tensor([  101,  6974,  1999,  1996,  3007,  9857,  1010,  1037, 25131,  5968,
         2730,  2019,  8956, 14460,  1010,  2021,  2019,  8956,  5239,  2704,
         1010, 10514, 10932,  2721, 14863,  2094, 14855, 10354,  2906,  1010,
         2704,  1997, 13508,  1998,  9230,  6376,  4895, 24572,  2102,  2013,
         2178,  5968,  8479,  1012,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

# Build model

In [ ]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased',
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(2.0726, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 8])

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

# Model training

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 2.0626039505004883
Training loss epoch: 0.8424169443109456
Training accuracy epoch: 0.7648783495502931
Training epoch: 2
Training loss per 100 training steps: 0.12671776115894318
Training loss epoch: 0.152384592768024
Training accuracy epoch: 0.8509327968051358
Training epoch: 3
Training loss per 100 training steps: 0.06994136422872543
Training loss epoch: 0.10672744966166861
Training accuracy epoch: 0.859344683150339
Training epoch: 4
Training loss per 100 training steps: 0.06969150900840759
Training loss epoch: 0.07645464009221863
Training accuracy epoch: 0.904667838713258
Training epoch: 5
Training loss per 100 training steps: 0.07127910107374191
Training loss epoch: 0.05524624637602007
Training accuracy epoch: 0.9412576139452585
Training epoch: 6
Training loss per 100 training steps: 0.036783602088689804
Training loss epoch: 0.04154103638275581
Training accuracy epoch: 0.9615750841219177
Training epoch: 7
Training loss per 100

# Model evaluation
Comments in 'Result analysis and Further Scope' header

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.011867574416100979
Validation Loss: 0.02253002385773203
Validation Accuracy: 0.9648208658986523


In [ ]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

                 precision    recall  f1-score   support

             SN       1.00      1.00      1.00        11
            ame       0.64      0.82      0.72        22
         ddress       0.00      0.00      0.00         2
    hone_number       0.50      1.00      0.67         1
          lates       0.00      0.00      0.00         4
           mail       0.95      0.91      0.93        22
reditCardNumber       1.00      1.00      1.00         2

      micro avg       0.71      0.81      0.76        64
      macro avg       0.59      0.68      0.62        64
   weighted avg       0.76      0.81      0.78        64



/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CreditCardNumber seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Name seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Plates seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Email seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: SSN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_l

# Predictions on test data + Post-processing
Comments in 'Result analysis and Further Scope' header

In [ ]:
test_df = pd.read_csv('PII Test Data.csv')
test_df

,Text,Label,PII
0,Term although process suddenly parent. Poor go...,NaN,NaN
1,"356 Collins Highway New Kathleen, NM 10160 Rem...",NaN,NaN
2,Appear job opportunity job. Piece 405 Callahan...,NaN,NaN
3,During half leave simple west lose piece 859 D...,NaN,NaN
4,Peace when Apt. 910 enter left speak agree. Le...,NaN,NaN
...,...,...,...
15995,He glass career yard community occur each. Bac...,NaN,NaN
15996,Everything religious home likely attention. Co...,NaN,NaN
15997,Matter adult look president than although road...,NaN,NaN
15998,Social require avoid book seven court decade.,NaN,NaN


In [ ]:
first_sentence = test_df['Text'][6000]
first_sentence

'Response enough boy hundred ability. Main board base Robert Reynolds religious meeting thus everything. Thank threat amount finish action detail.'

In [ ]:
def predict(first_sentence):

  inputs = tokenizer(first_sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

  # move to gpu
  ids = inputs["input_ids"].to(device)
  mask = inputs["attention_mask"].to(device)
  # forward pass
  outputs = model(ids, mask)
  logits = outputs[0]

  active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
  flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

  tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
  token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
  wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

  word_level_predictions = []
  for pair in wp_preds:
    if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
      # skip prediction
      continue
    else:
      word_level_predictions.append(pair[1])

  # we join tokens, if they are not special ones
  orig_sentence_split = [t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]
  # print (orig_sentence_split)
  str_rep = " ".join(orig_sentence_split).replace(" ##", "")

  #post-process to reduce false positives
  prep_df = pd.DataFrame({'Label': word_level_predictions, 'Word': orig_sentence_split})
  uniq_labels = prep_df['Label'].unique().tolist()

  label_list = []
  pii_list = []
  if 'CreditCardNumber' in uniq_labels:
    combined_string = ''.join([value.replace('##', '') for value in prep_df.loc[prep_df['Label'] == 'CreditCardNumber', 'Word']])
    label_list.append('CreditCardNumber')
    pii_list.append(combined_string)

  if 'Email' in uniq_labels:
    combined_string = ''.join([value.replace('##', '') for value in prep_df.loc[prep_df['Label'] == 'Email', 'Word']])
    if re.match(r'^[\w\.-]+@[\w\.-]+\.\w+$', combined_string):
      label_list.append('Email')
      pii_list.append(combined_string)

  if 'Name' in uniq_labels:
    combined_string = ' '.join([value.replace('##', '') for value in prep_df.loc[prep_df['Label'] == 'Name', 'Word']])
    label_list.append('Name')
    pii_list.append(combined_string)

  if 'Phone_number' in uniq_labels:
    combined_string = ''.join([value.replace('##', '') for value in prep_df.loc[prep_df['Label'] == 'Phone_number', 'Word']])
    label_list.append('Phone_number')
    pii_list.append(combined_string)

  if 'Plates' in uniq_labels:
    combined_string = ''.join([value.replace('##', '') for value in prep_df.loc[prep_df['Label'] == 'Plates', 'Word']])
    label_list.append('Plates')
    pii_list.append(combined_string)

  if 'SSN' in uniq_labels:
    combined_string = ''.join([value.replace('##', '') for value in prep_df.loc[prep_df['Label'] == 'SSN', 'Word']])
    if len(combined_string) >= 9:
      label_list.append('SSN')
      pii_list.append(combined_string)

  if 'Address' in uniq_labels:
    combined_string = ' '.join([value.replace('##', '') for value in prep_df.loc[prep_df['Label'] == 'Address', 'Word']])
    label_list.append('Address')
    pii_list.append(combined_string)

  return label_list, pii_list

for idx, row in test_df.iterrows():
  label_list, pii_list = predict(row['Text'])
  test_df.loc[idx,'Label'] = str(label_list)
  test_df.loc[idx,'PII'] = str(pii_list)



In [ ]:
test_df.to_csv('predictions_testdata_postchecks.csv')

# Result analysis and Further Scope

1. A general rule to improve f1-score across entities is to have more data, as we fine-tuned on around 10 examples per entity
2. <b>Name : </b> Name entity was initially not being identified at all as it requires contexual information as well. Adding additional data from NER dataset(+100 data points) specifically for Name improved the results significantly, proving the first point
3. <b>Email ID : </b> While Email ID was being identified well, there were many false positives where we taken care by validating using regex
4. <b>SSN : </b> While SSN was being identified well, there were many false positives where we taken care by validating the length as there is a standard length of the digits
5. <b>CreditCardNumber : </b> While this was mostly captured, it was often misclassified as PhoneNumber. A simple regex would not fix this as it is possible to have same format, while more contextually aware sentences would make sense
6. <b>PhoneNumber : </b> While this was mostly captured, it was often identified as CreditCardNumber. A simple regex would not fix this as it is possible to have same format, while more contextually aware sentences would make sense
7. <b>Address : </b> This was a difficult entity as it consisted of more than 1 component. Parts of it were mostly identified, but not the whole entity as such. One way to make this work, is to create sentences with the components as separate entities. For example, '00779 Nicole Mountain Apt. 347 North Veronicaborough, TN 58877' can be further broken down into<br>
Street Number: 00779<br>
Street Name: Nicole Mountain<br>
Apartment Number: Apt. 347<br>
City: North Veronicaborough<br>
State: TN<br>
Postal Code: 58877<br>
There are address parsing libraries which can do this

Another solution is https://lajavaness.medium.com/1-token-classification-vs-span-categorization-52a685e4674a where Span Categorization is based on multilabel classification at the token level (to remind the difference, see here). To convert multiclass classification into multilabel classification, it suffices to change:
The loss function: Cross-entropy loss -> Binary cross-entropy loss and The activation at the output layer of BERT model: Softmax -> Sigmoid.

8. <b>Plates : </b> This also was a difficult entity as the subwords tokens were mostly classifying as EmailID+Address+Plates. It would definitely improve with more data as regex can be difficult to validate further because of the many variations possible.Also, false positives of other entities would decrease.